# EM shower reconstruction at SND@LHC

1. __make sure the preprocessing has already been done__

2. __make sure `results` folder exists__

https://arxiv.org/pdf/2002.08722.pdf

In [ ]:
#for running all notebooks use e.g.
# jupyter nbconvert --to notebook --inplace --execute regression_*.ipynb

In [1]:
# imports from utils.py & net.py
from utils import DataPreprocess, Parameters
#from net import SNDNet, BNN, MyDataset, digitize_signal, digitize_signal_1d

# python
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pylab as plt
import time
from tqdm import tqdm
from IPython import display

# system
import os
import gc  # Gabage collector interface (to debug stuff)
import sys

# ml
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# dl
import torch
import torch.nn as nn

Welcome to JupyROOT 6.18/00


In [2]:
# Test to see if cuda is available or not + listed the CUDA devices that are available
try:
    assert(torch.cuda.is_available())
except:
    raise Exception("CUDA is not available")
    
n_devices = torch.cuda.device_count()
print("CUDA devices available:")

for i in range(n_devices):
    print("\t{}\twith CUDA capability {}".format(torch.cuda.get_device_name      (device=i), 
                                                 torch.cuda.get_device_capability(device=i)))

device = torch.device("cuda", 0)

CUDA devices available:
	Quadro RTX 4000	with CUDA capability (7, 5)
	Quadro RTX 4000	with CUDA capability (7, 5)


In [3]:
# Turn off interactive plotting: for long run it screws up everything
plt.ioff()

In [4]:
DETECTOR_PARAMS = Parameters("SNDatLHC")
DETECTOR_CONFIG = DETECTOR_PARAMS.snd_params[DETECTOR_PARAMS.configuration]

# number of planes of the detector
#NB_PLANE = dict()

#NB_PLANE['scifi']   = len(DETECTOR_CONFIG['SciFi_tracker']        ['TT_POSITIONS'])
#NB_PLANE['up_mu']   = len(DETECTOR_CONFIG['Mu_tracker_upstream']  ['TT_POSITIONS'])
#NB_PLANE['down_mu'] = len(DETECTOR_CONFIG['Mu_tracker_downstream']['TT_POSITIONS'])

## Data processing

Here we load and process __pickle__ files. 

In [5]:
from src.process_pickle import *

In [6]:
DATA_PATH = dict()
DATA_PATH['nuel']  = "~/snd_data/nue"
DATA_PATH['numu']  = "~/snd_data/numu"
DATA_PATH['nutau'] = "~/snd_data/nutau"

EVENTS_PER_FILE = 4000 # todo -> read from the files ?
FILES_NUM       = 10   # MAX=100 / todo -> read from directory ?

In [7]:
#scifi_arr, mu_arr, en_arr = load_dataframes(DETECTOR_PARAMS, 
#                                            DATA_PATH, EVENTS_PER_FILE, FILES_NUM)

In [8]:
#scifi_arr, mu_arr, en_arr = merge_events_arrays(scifi_arr, mu_arr, en_arr)

In [9]:
#en_arr = normalise_target_energy(en_arr)

## Data preparation

Here we prepare (load or, if needed, create) the datasets.

In [10]:
from src.operate_datasets import *

In [11]:
#create_dataset('true', DETECTOR_PARAMS, DATA_PATH, EVENTS_PER_FILE, FILES_NUM)

In [12]:
# create_dataset('sum', DETECTOR_PARAMS, DATA_PATH, EVENTS_PER_FILE, FILES_NUM)

In [13]:
# create_dataset('longitudal', DETECTOR_PARAMS, DATA_PATH, EVENTS_PER_FILE, FILES_NUM)

In [14]:
# create_dataset('projection', DETECTOR_PARAMS, DATA_PATH, EVENTS_PER_FILE, FILES_NUM)

In [15]:
# memory troubles!
# be very carefull when using this
### create_dataset('plane', DETECTOR_PARAMS, DATA_PATH)

## Models

### Support vector regression baseline

In [16]:
full_X, full_y = load_dataset('~/snd_data/new_dataset/', 'longitudal')

X_train, y_train, _, _ = split_dataset(full_X, full_y)
# min_clip = 25
# X_train, y_train = clip_dataset(X_train, y_train, min_clip)

In [ ]:
from sklearn import svm

reg_svr = svm.SVR(gamma='scale')
#reg_svr = svm.LinearSVR(max_iter=10**5)

reg_svr.fit(X_train, y_train)

score_svr = reg_svr.score(X_train, y_train)

print('SVM: ', score_svr)

y_pred_svr = reg_svr.predict(X_train)

In [ ]:
X_sum = X_train.sum(axis=1).reshape(-1,1)
y_sum = y_train.reshape(-1,1)
y_pred_svr = y_pred_svr.reshape(-1,1)

In [ ]:
plot_res_vs_energy(X_sum, y_sum, y_pred_svr) 

In [ ]:
plot_res_hist(y_sum, y_pred_svr)

In [ ]:
plot_2d_energy_hist(X_sum, y_sum, y_pred_svr)

In [ ]:
get_scores(y_sum, y_pred_svr)

In [ ]:
frac_resol = np.divide((y_pred_svr - y_sum), y_sum)

print((frac_resol < 0).sum())
print((frac_resol >= 0).sum())

### Stochastic regression

In [ ]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import SVI, Trace_ELBO, Predictive

In [ ]:
full_X, full_y = load_dataset('~/snd_data/new_dataset/', 'sum')
min_clip = 25

X_train, y_train = dataset_split(full_X, full_y)
X_train, y_train = dataset_clip(X_train, y_train, min_clip)

In [ ]:
X_train = torch.tensor(X_train).float().reshape(-1,1)
y_train = torch.tensor(y_train).float().reshape(-1)

In [ ]:
from pyro.distributions import constraints


class BayesianRegression(PyroModule):
    def __init__(self):
        super().__init__()
        self.linear = PyroModule[nn.Linear](1, 1)
        
        # prior over parameters
        self.linear.weight = PyroSample(dist.Normal(1e-4, 5e-5).expand([1, 1]).to_event(2))
        self.linear.bias   = PyroSample(dist.Normal(1e-1, 5e-2).expand([1]).to_event(1))
        return
    
    def forward(self, x, y=None):
        # energy (from the model)
        mean = self.linear(x).squeeze(-1)
        
        # noise (learnable)
        sigma = pyro.sample("sigma", dist.Uniform(0., 1.)) 
        
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mean, sigma), obs=y)
        
        return mean

In [ ]:
def model(x, y=None):
    bias   = pyro.sample("bias",   dist.Normal(1e-4, 1e-4))
    weight = pyro.sample("weight", dist.Normal(0.1, 0.1))

    mean = bias + weight * x # + mu_weight * muon_hits
    mean = mean.squeeze(-1)
    sigma = pyro.sample("sigma", dist.Uniform(0., 0.1)) 

    with pyro.plate("data", x.shape[0]):
        pyro.sample("obs", dist.Normal(mean, sigma), obs=y)

    return mean


def guide(x, y=None):
    b_loc   = pyro.param("b_loc",   torch.tensor(1e-4), constraint=constraints.positive)
    b_scale = pyro.param("b_scale", torch.tensor(1e-4), constraint=constraints.positive)

    w_loc   = pyro.param("w_loc",   torch.tensor(0.1), constraint=constraints.positive)
    w_scale = pyro.param("w_scale", torch.tensor(0.1), constraint=constraints.positive)

    bias =   pyro.sample("bias",   dist.Normal(b_loc, b_scale))
    weight = pyro.sample("weight", dist.Normal(w_loc, w_scale))

    #sigma_loc = pyro.param('sigma_loc', torch.tensor(5.))
    sigma = pyro.sample("sigma", dist.Normal(0.05, 0.005))

    mean = bias + weight * x
    
    return mean

In [ ]:
reg_model = model # BayesianRegression()
reg_guide = guide # AutoDiagonalNormal(model)

num_steps = 50
initial_lr = 1.0
gamma = 0.5  # final learning rate will be gamma * initial_lr
lrd = gamma ** (1 / num_steps)

adam = pyro.optim.ClippedAdam({"lr": initial_lr, 'lrd': lrd})

#adam = pyro.optim.ClippedAdam({"lr": 1.0, "lrd": 0.5})
svi = SVI(reg_model, reg_guide, adam, loss=Trace_ELBO())
num_iterations = 200

pyro.clear_param_store()
loss_arr = []


for j in range(num_iterations):
    loss = svi.step(X_train, y_train)

    loss_arr.append(loss)
        
    if j % 25 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(X_train)))

In [ ]:
plt.plot(loss_arr)
plt.yscale('log')
plt.xlabel('epoch')
plt.ylabel('ELBO loss')
plt.show()

In [ ]:
#guide.requires_grad_(False)

for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

In [ ]:
def summary(samples):
    site_stats = {}
    for site_name, values in samples.items():
        site_stats[site_name] = {
            "mean": torch.mean(values, 0),
            "std" : torch.std (values, 0),
            "5%"  : values.kthvalue(int(len(values) * 0.05), dim=0)[0],
            "95%" : values.kthvalue(int(len(values) * 0.95), dim=0)[0],
        }
        
    return site_stats


predictive = Predictive(model, guide=guide, num_samples=800)
samples = predictive(X_train)
pred_summary = summary(samples)

In [ ]:
pred_summary

In [ ]:
mu = pred_summary["obs"]
y = pred_summary["obs"]
predictions = pd.DataFrame({
    "mu_mean": mu["mean"],
    "mu_perc_5": mu["5%"],
    "mu_perc_95": mu["95%"],
})

In [ ]:
plt.plot(X_train[:,0], predictions['mu_mean'].to_numpy(), 'o')
plt.plot(X_train[:,0], predictions['mu_perc_5'].to_numpy(), 'o')
plt.plot(X_train[:,0], predictions['mu_perc_95'].to_numpy(), 'o')
plt.show()

In [ ]:
predictions['mu_mean']

In [ ]:
def plot_2d_energy_hist(X_arr, y_true, predictions):
    fig, ax = plt.subplots(figsize=(8,6))

    y_pred_min  = predictions["mu_perc_5"].to_numpy()
    y_pred_mean = predictions["mu_mean"]  .to_numpy()
    y_pred_max  = predictions["mu_perc_95"].to_numpy()
    
    print(X_arr.shape)
    print(y_true.shape)
    
    hist = ax.hist2d(X_arr[:,0].numpy(), y_true.numpy(), 
                     bins=100, norm=mpl.colors.LogNorm(), vmax=150)
    
    plt.ylim(0)
    plt.xlabel('pixel sum')
    plt.ylabel('normalised energy')

    #plt.axvline(x=min_clip, c='m', alpha=0.9, label='Min clip ' + str(min_clip))
    ax.plot(X_arr[:, 0], y_pred_mean, 'g.', alpha=0.3, label='L2 w. clipped data')
    ax.plot(X_arr[:, 0], y_pred_min, 'r.', marker='.', alpha=0.3, label='5%')
    ax.plot(X_arr[:, 0], y_pred_max, 'b.', marker='.', alpha=0.3, label='95%')
    #ax.fill_between(X_arr[:, 0], y_pred_min, y_pred_max,  alpha=0.3, color='deeppink')

    #cbar = fig.colorbar(hist[3], ax=ax)
    #|cbar.set_label('# of particles')

    plt.legend(loc='lower right')
    plt.show()

In [ ]:
plot_2d_energy_hist(X_train, y_train, predictions) 